# Lab 9: Design of Experiment
In this lab, we will observe the effect of the DoE in the Bayesian optimization and Bio-inspired approaches.

In the following (hidden) block, the utilities used for running the experiments are implemented.

The list of available benchmark functions can be found at this [link](https://gitlab.com/luca.baronti/python_benchmark_functions)

**NOTE**: When studying the effect of the parameters is *extremely* important to vary just one parameter at a time. Therefore, you are suggested to study one parameter by fixing all the others, and then moving to the next.

Moreover, when comparing different algorithms, is *very* important to run each of them several times (e.g., 30) by using different initial random seeds.


You will use the sampling technique seen in the lecture. In particular, you will have to implement:


*   Random sampling
*   The Halton sequence
*   The full factorial sampling
*   The Latin Hypercube sampling



Code for the Bayesian Optimization, you have to reuse the code from Lab. 6 for the acquisition and the objective functions.

In [ ]:
import itertools as it
import shutil
from builtins import float
from typing import Iterator, Union, Tuple, Callable, List
from warnings import catch_warnings, simplefilter

import more_itertools
import numpy as np
import pipe as p
import scipy as sp
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
def objective(x: float, noise: float = 0.1) -> float:
    return x ** 2 * np.sin(8 * np.pi * x) + np.random.uniform(-noise, noise)

In [ ]:
# region bo
XS = np.arange(0, 1, 0.001).reshape(-1, 1)
YS = objective(XS, 0)


def surrogate(model: GaussianProcessRegressor, X: np.ndarray[float]) -> Tuple[np.ndarray[float], np.ndarray[float]]:
    """
    surrogate or approximation for the objective function
    """

    # catch any warning generated when making a prediction
    with catch_warnings():
        # ignore generated warnings
        simplefilter("ignore")
        return model.predict(X, return_std=True)


def opt_acquisition(
        rnd: np.random.Generator,
        xs_known: np.ndarray[float],
        ys_known: np.ndarray[float],
        model: GaussianProcessRegressor,
        acquisition: Callable[[np.ndarray[float], np.ndarray[float], GaussianProcessRegressor], np.ndarray[float]]
) -> float:
    """
    optimize the acquisition function
    """

    # random search, generate random samples
    xs_unknown: np.ndarray[float] = rnd.random(50)
    xs_unknown = xs_unknown.reshape(-1, 1)

    # calculate the acquisition function for each sample
    scores = acquisition(xs_known, xs_unknown, model)

    # locate the index of the largest scores
    ix = np.argmax(scores)
    return xs_unknown[ix, 0]


def bayesian_optimization(
        rnd: np.random.Generator,
        generation: int,
        model: GaussianProcessRegressor,
        acquisition: Callable[[np.ndarray[float], np.ndarray[float], GaussianProcessRegressor], np.ndarray[float]],
        initial_points: np.ndarray[np.ndarray[float]],
        file: str
) -> Tuple[np.ndarray[float], np.ndarray[float], GaussianProcessRegressor]:
    # reshape into rows and cols
    xs_known = initial_points.reshape(-1, 1)
    ys_known = np.asarray([objective(x) for x in xs_known]).reshape(-1, 1)

    # fit the model
    model.fit(xs_known, ys_known)

    # perform the optimization process
    for i in range(generation):
        # select the next point
        # and sample it
        x_next = opt_acquisition(rnd, xs_known, ys_known, model, acquisition)
        y_next = objective(x_next)

        # region plot
        fig, (ax1, ax2) = plt.subplots(2, sharex=True, height_ratios=[3, 1], gridspec_kw={'hspace': 0})
        plot_approximation(ax1, model, xs_known, ys_known, x_next)
        plot_acquisition(ax2, model, xs_known, acquisition, x_next)
        if i == 0:
            fig.legend(loc='upper left')
        fig.savefig(f'{file} {i}.svg')
        plt.close('all')
        # endregion

        # add the data to the dataset
        xs_known = np.vstack((xs_known, [[x_next]]))
        ys_known = np.vstack((ys_known, [[y_next]]))

        # update the model
        model.fit(xs_known, ys_known)

    return xs_known, ys_known, model

# endregion

In [ ]:
# region bo plots
def plot_approximation(
        ax,
        model,
        xs_known,
        ys_known,
        x_next,
):
    mu, std = model.predict(XS, return_std=True)
    ax.fill_between(
        XS.ravel(),
        mu.ravel() + 1.96 * std,
        mu.ravel() - 1.96 * std,
        alpha=0.1
    )
    ax.fill_between(
        XS.ravel(),
        YS.ravel() + 0.1,
        YS.ravel() - 0.1,
        alpha=0.1
    )
    ax.plot(XS, YS, 'y--', lw=1, label='objective')
    ax.plot(XS, mu, 'b-', lw=1, label='surrogate function')
    ax.plot(xs_known, ys_known, 'kx', mew=3, label='noisy samples')
    ax.axvline(x=x_next, ls='--', c='k', lw=1)


def plot_acquisition(
        ax,
        model,
        xs_known,
        acquisition: Callable[[np.ndarray[float], np.ndarray[float], GaussianProcessRegressor], np.ndarray[float]],
        x_next,
):
    ax.plot(XS, acquisition(xs_known, XS, model), 'r-', lw=1, label='Acquisition function')
    ax.axvline(x=x_next, ls='--', c='k', lw=1, label='Next sampling location')

# endregion

In [ ]:
# region bo implementations
def probability_of_improvement(
        xs_known: np.ndarray[float],
        xs_unknown: np.ndarray[float],
        model: GaussianProcessRegressor
) -> np.ndarray[float]:
    y_hat, _ = surrogate(model, xs_known)
    best = max(y_hat)

    mu, sd = surrogate(model, xs_unknown)
    z = (best - mu) / sd

    return sp.stats.norm.cdf(-z)


def expected_improvement(
        xs_known: np.ndarray[float],
        xs_unknown: np.ndarray[float],
        model: GaussianProcessRegressor,
) -> np.ndarray[float]:
    y_hat, _ = surrogate(model, xs_known)
    best = max(y_hat)

    mu, sd = surrogate(model, xs_unknown)
    z = (best - mu) / sd

    return (mu - best) * sp.stats.norm.cdf(-z) + sd * sp.stats.norm.pdf(-z)

# endregion

Code for the Genetic algorithm from lab. 7

In [ ]:
from pathlib import Path
from pylab import *
from inspyred import ec
from copy import deepcopy

import benchmark_functions as bf

In [ ]:
# region ga
GLOBAL = 'Global'
INDIVIDUAL = 'Individual'
CORRELATED = 'Correlated'
STAR = 'star'
RING = 'ring'


class OptFun():
    def __init__(self, wf):
        self.f = wf
        self.history = []
        self.__name__ = f'OptFun({wf.__class__})'

    def __call__(self, candidates, *args, **kwargs):
        y = []
        for x0 in candidates:
            self.history.append(deepcopy(x0))
            y.append(self.f(x0))
        return y

    def minima(self):
        return self.f.minima()

    def bounder(self):
        def fcn(candidate, *args):
            bounds = self.f.suggested_bounds()

            for i, (m, M) in enumerate(zip(*bounds)):
                if candidate[i] < m:
                    candidate[i] = m
                if candidate[i] > M:
                    candidate[i] = M
            return candidate

        return fcn

    def bounds(self):
        return self._convert_bounds(self.f.suggested_bounds())

    def heatmap(self, file=None):
        plt.clf()
        resolution = 50
        fig = plt.figure()
        bounds_lower, bounds_upper = self.f.suggested_bounds()
        x = np.linspace(bounds_lower[0], bounds_upper[0], resolution)
        y = np.linspace(bounds_lower[1], bounds_upper[1], resolution)
        X, Y = np.meshgrid(x, y)
        Z = np.asarray([[self.f((X[i][j], Y[i][j])) for j in range(len(X[i]))] for i in range(len(X))])

        plt.contour(x, y, Z, 15, linewidths=0.5, colors='k')  # height lines
        plt.contourf(x, y, Z, 15, cmap='viridis', vmin=Z.min(), vmax=Z.max())  # heat map
        plt.xlabel('x')
        plt.ylabel('y')
        cbar = plt.colorbar()
        cbar.set_label('z')
        if len(self.history) > 0:  # plot points
            xdata = [x for [x, _] in self.history]
            ydata = [y for [_, y] in self.history]
            plt.plot(xdata, ydata, 'or', markersize=3)
        if file is None:
            plt.show()
        else:
            fig.savefig(file, dpi=400)
        plt.close('all')

    def heatmapP(self, window: int, init: int = 0, file=None):
        plt.clf()
        resolution = 50
        fig = plt.figure()
        bounds_lower, bounds_upper = self.f.suggested_bounds()
        x = np.linspace(bounds_lower[0], bounds_upper[0], resolution)

        y = np.linspace(bounds_lower[1], bounds_upper[1], resolution)
        X, Y = np.meshgrid(x, y)
        Z = np.asarray([[self.f((X[i][j], Y[i][j])) for j in range(len(X[i]))] for i in range(len(X))])

        plt.contour(x, y, Z, 15, linewidths=0.5, colors='k')  # height lines
        plt.contourf(x, y, Z, 15, cmap='viridis', vmin=Z.min(), vmax=Z.max())  # heat map
        plt.xlabel('x')
        plt.ylabel('y')
        cbar = plt.colorbar()
        cbar.set_label('z')
        if len(self.history) > 0:  # plot points

            plt.plot(
                [x for [x, _] in self.history[:init]],
                [y for [_, y] in self.history[:init]],
                'ow',
                markersize=3,
                alpha=0.5
            )

            list(self.history[init:]
                 | p.Pipe(lambda ps: more_itertools.windowed(ps, n=window, step=window))
                 | p.izip(it.count(1))
                 | p.map(lambda ps: plt.plot([x for [x, _] in ps[0]], [y for [_, y] in ps[0]], 'or', markersize=3,
                                             alpha=0.1 + 0.9 * (ps[1] / (len(self.history) / window))))
                 )

        if file is None:
            plt.show()
        else:
            fig.savefig(file, dpi=400)
        plt.close('all')

    def plot(self, minima=None, file=None):
        plt.clf()
        values = [self.f(v) for v in self.history]
        m = min(
            [np.Inf if minima is None else minima] +
            [self.f.minimum().score] +
            list(self.f.minima() | p.map(lambda x: x.score))
        )
        plt.plot(values)
        plt.axhline(m, color="r", label="optimum")
        plt.legend()
        if file is None:
            plt.show()
        else:
            plt.savefig(file, dpi=400)
        plt.close('all')

    def _convert_bounds(self, bounds):
        new_bounds = []
        for i in range(len(bounds[0])):
            new_bounds.append((bounds[0][i], bounds[1][i]))
        return new_bounds

    def current_calls(self):
        return len(self.history)


def choice_without_replacement(rng: np.random.RandomState, n, size):
    return rng.choice(range(0, n), size=size, replace=False)


class NumpyRandomWrapper(np.random.RandomState):
    def __init__(self, seed=None):
        super(NumpyRandomWrapper, self).__init__(seed)

    def sample(self, population, k):
        if isinstance(population, int):
            population = range(population)

        return np.asarray([
            population[i]
            for i
            in choice_without_replacement(self, len(population), k)
        ])
        #return #self.choice(population, k, replace=False)

    def random(self):
        return self.random_sample()

    def gauss(self, mu, sigma):
        return self.normal(mu, sigma)


def initial_pop_observer(
        population,
        num_generations,
        num_evaluations,
        args
):
    if num_generations == 0:
        args["initial_pop_storage"]["individuals"] = np.asarray([guy.candidate for guy in population])
        args["initial_pop_storage"]["fitnesses"] = np.asarray([guy.fitness for guy in population])


def generator_wrapper(func):
    @functools.wraps(func)
    def _generator(random, args):
        return np.asarray(func(random, args))

    return _generator


def single_objective_evaluator(candidates, args):
    problem = args["problem"]
    return [CombinedObjectives(fit, args) for fit in
            problem.evaluator(candidates, args)]


def run_ga(
        rnd: np.random.RandomState,
        func,
        **kwargs
):
    #create dictionaries to store data about initial population, and lines
    initial_pop_storage = {}

    algorithm = ec.EvolutionaryComputation(rnd)
    algorithm.terminator = ec.terminators.generation_termination
    algorithm.replacer = ec.replacers.generational_replacement
    algorithm.variator = [ec.variators.uniform_crossover, ec.variators.gaussian_mutation]
    algorithm.selector = ec.selectors.tournament_selection

    algorithm.observer = initial_pop_observer

    kwargs["num_selected"] = kwargs["pop_size"]
    kwargs["bounder"] = func.bounder()

    final_pop = algorithm.evolve(
        evaluator=func,
        maximize=False,
        generator=None,
        initial_pop_storage=initial_pop_storage,
        num_var=2,
        **kwargs
    )

    #best_guy = final_pop[0].candidate
    #best_fitness = final_pop[0].fitness
    final_pop_fitnesses = np.asarray([guy.fitness for guy in final_pop])
    final_pop_candidates = np.asarray([guy.candidate for guy in final_pop])

    sort_indexes = sorted(range(len(final_pop_fitnesses)), key=final_pop_fitnesses.__getitem__)
    final_pop_fitnesses = final_pop_fitnesses[sort_indexes]
    final_pop_candidates = final_pop_candidates[sort_indexes]

    best_guy = final_pop_candidates[0]
    best_fitness = final_pop_fitnesses[0]

    return best_guy, best_fitness, final_pop

# endregion

Samplings methods to implement

In [ ]:
def random_generator(
        boundaries: List[Tuple[int, int]],
        pop: int,
        rnd: np.random.Generator,
) -> np.ndarray[np.ndarray[float]]:
    return np.asarray([rnd.uniform(low=start, high=stop, size=pop) for (start, stop) in boundaries]).T


def lhs_generator(
        boundaries: List[Tuple[int, int]],
        pop: int,
        rnd: np.random.Generator,
) -> np.ndarray[np.ndarray[float]]:
    n = len(boundaries)

    # generate the intervals
    cut = np.linspace(0, 1, pop + 1)

    # fill points uniformly in each interval
    u = rnd.random([pop, n])
    a = cut[:pop]
    b = cut[1:pop + 1]
    rnd_points = np.zeros_like(u)
    for j in range(n):
        rnd_points[:, j] = u[:, j] * (b - a) + a

    # make the random pairings
    h = np.zeros_like(rnd_points)
    for j in range(n):
        order = rnd.permutation(range(pop))
        h[:, j] = rnd_points[order, j]

    h = h.T

    for (i, (start, stop)) in zip(it.count(0), boundaries):
        h[i] = start + (h[i] * (stop - start))

    return h.T


# ppf_start, ppf_stop = (-2, 2)

# if len(boundaries) == 1:
#     [(start, stop)] = boundaries
#     return start + (abs(ppf_start) + sp.stats.norm.ppf(
#         [
#             (i + 1 - rnd.uniform()) / pop
#             for i
#             in rnd.permutation(pop)
#         ]
#     )) * (stop - start) / (ppf_stop - ppf_start)

# return np.asarray(
#     [
#         start + (abs(ppf_start) + sp.stats.norm.ppf(
#             [
#                 (i + 1 - rnd.uniform()) / pop
#                 for i
#                 in rnd.permutation(pop)
#             ]
#         )) * (stop - start) / (ppf_stop - ppf_start)
#         for (start, stop)
#         in boundaries
#     ]
# ).T


def halton(
        start: int,
        stop: int,
        base: int
) -> Iterator[float]:
    n, d = 0, 1
    while True:
        x = d - n
        if x == 1:
            n = 1
            d *= base
        else:
            y = d // base
            while x <= y:
                y //= base
            n = (base + 1) * y - x
        yield start + ((n / d) * (stop - start))


def halton_generator(
        boundaries: List[Tuple[int, int]],
        pop: int,
        bases: List[int]
) -> np.ndarray[np.ndarray[float]]:
    return np.asarray([
        np.fromiter(iter=halton(start, stop, base), dtype=float, count=pop) for [(start, stop), base] in
        zip(boundaries, bases)
    ]).T


def ff_generator(
        boundaries: List[Tuple[int, int]],
        m: int
) -> np.ndarray[np.ndarray[float]]:
    return np.asarray(
        list(it.product(*[
            np.linspace(start=start, stop=stop, num=m)
            for (start, stop)
            in boundaries
        ]))
    )


In [ ]:
def compare() -> None:
    m = 50
    primes = [2, 3, 5, 7]

    for boundaries in [[(0, 10)]]:
        pop = m ** len(boundaries)
        gs = [
            ('uniform', random_generator(boundaries, pop, np.random.default_rng(seed=0))),
            ('lhs', lhs_generator(boundaries, pop, np.random.default_rng(seed=0))),
            (f'halton : {primes[:len(boundaries)]}', halton_generator(boundaries, pop, primes[:len(boundaries)])),
            ('full factorial', ff_generator(boundaries, m)),
        ]

        fig, axs = plt.subplots(
            nrows=2,
            ncols=2,
            figsize=(2 * 4, 3),
            dpi=400,
            sharex=True,
            sharey=True,
        )

        axs = it.chain.from_iterable(axs)

        for (i, ax, (g, ps)) in zip(it.count(1), axs, gs):
            ax.set_anchor('W')
            ax.scatter(
                ps,
                np.full(len(ps), 0),
                s=20,
                alpha=0.8,
                label=g,
                c=f'C{i}',
            )
            ax.set(
                aspect='auto',
                xlim=boundaries[0],
                xticks=[],
                yticks=[],
                autoscale_on=False,
            )
            ax.legend(loc='lower left')

        fig.subplots_adjust(wspace=0, hspace=0)
        fig.savefig('comparing/1D.svg')
        plt.close('all')

    for boundaries in [[(0, 10), (0, 10)]]:
        pop = m ** len(boundaries)
        gs = [
            ('uniform', random_generator(boundaries, pop, np.random.default_rng(seed=0))),
            ('lhs', lhs_generator(boundaries, pop, np.random.default_rng(seed=0))),
            (f'halton : {primes[:len(boundaries)]}', halton_generator(boundaries, pop, primes[:len(boundaries)])),
            ('full factorial', ff_generator(boundaries, m)),
        ]

        fig, axs = plt.subplots(
            nrows=2,
            ncols=2,
            figsize=(2 * 4, 2 * 4),
            dpi=400,
            sharex=True,
            sharey=True,
        )

        axs = it.chain.from_iterable(axs)

        for (i, ax, (g, ps)) in zip(it.count(1), axs, gs):
            ax.set_anchor('W')
            ax.scatter(
                ps.T[0],
                ps.T[1],
                s=20,
                alpha=0.8,
                label=g,
                c=f'C{i}',
            )
            ax.set(
                aspect='auto',
                xlim=boundaries[0],
                ylim=boundaries[1],
                xticks=[],
                yticks=[],
                autoscale_on=False,
            )
            ax.legend(loc='lower left')

        fig.subplots_adjust(wspace=0, hspace=0)
        fig.savefig('comparing/2D.svg')
        plt.close('all')

    m = 10
    for boundaries in [[(0, 10), (0, 10), (0, 10)]]:
        pop = m ** len(boundaries)
        gs = [
            ('uniform', random_generator(boundaries, pop, np.random.default_rng(seed=0))),
            ('lhs', lhs_generator(boundaries, pop, np.random.default_rng(seed=0))),
            (f'halton : {primes[:len(boundaries)]}', halton_generator(boundaries, pop, primes[:len(boundaries)])),
            ('full factorial', ff_generator(boundaries, m)),
        ]

        fig, axs = plt.subplots(
            nrows=2,
            ncols=2,
            figsize=(2 * 4, 2 * 4),
            dpi=400,
            subplot_kw={'projection': '3d'},
        )

        axs = it.chain.from_iterable(axs)

        for (i, ax, (g, ps)) in zip(it.count(1), axs, gs):
            ax.set_anchor('W')
            ax.scatter(
                ps.T[0],
                ps.T[1],
                ps.T[2],
                s=20,
                alpha=0.8,
                label=g,
                c=f'C{i}',
            )
            ax.set(
                aspect='auto',
                xlim=boundaries[0],
                ylim=boundaries[1],
                zlim=boundaries[2],
                xticks=[],
                yticks=[],
                zticks=[],
                autoscale_on=False,
            )
            ax.legend(loc='lower left')

        fig.subplots_adjust(wspace=0, hspace=0)
        fig.savefig('comparing/3D.svg')
        plt.close('all')

        for (i, (g, ps)) in zip(it.count(1), gs):
            fig, axs = plt.subplots(
                nrows=2,
                ncols=2,
                figsize=(2 * 4, 2 * 4),
                dpi=400,
            )

            fig.delaxes(axs[-1][-1])

            for (ax, (r, c)) in zip(it.chain.from_iterable(axs), [(0, 1), (2, 1), (0, 2)]):
                ax.set_anchor('W')
                ax.scatter(
                    ps.T[r],
                    ps.T[c],
                    s=20,
                    alpha=0.8,
                    c=f'C{i}',
                )
                ax.set(
                    aspect='auto',
                    xlim=boundaries[r],
                    ylim=boundaries[c],
                    xticks=[],
                    yticks=[],
                    autoscale_on=False,
                )

            fig.subplots_adjust(wspace=0, hspace=0)
            fig.savefig(f'comparing/3D - {g}.svg')
            plt.close('all')


shutil.rmtree('comparing')
Path('comparing').mkdir(parents=True, exist_ok=False)
compare()

# Exercise 1: Bayesian Optimization

In [ ]:
def bo() -> None:
    shutil.rmtree('bo')
    Path('bo').mkdir(parents=True, exist_ok=False)

    m = 10
    primes = [2, 3, 5, 7]
    boundaries = [(0, 1)]
    pop = m ** len(boundaries)
    gs = [
        ('uniform', random_generator(boundaries, pop, np.random.default_rng(seed=0))),
        ('lhs', lhs_generator(boundaries, pop, np.random.default_rng(seed=0))),
        (f'halton : {primes[:len(boundaries)]}', halton_generator(boundaries, pop, primes[:len(boundaries)])),
        ('full factorial', ff_generator(boundaries, m)),
    ]

    for (g, ps) in gs:
        bayesian_optimization(
            rnd=np.random.default_rng(seed=0),
            generation=10,
            model=GaussianProcessRegressor(),
            acquisition=probability_of_improvement,
            initial_points=ps,
            file=f'bo/pi {g}'
        )


shutil.rmtree('bo')
Path('bo').mkdir(parents=True, exist_ok=False)
bo()

# Exercise 2: Genetic Algorithm

In [ ]:
bfs = [
    ("ackley", bf.Ackley(), None),
    ("de jong 3", bf.DeJong3(), None),
    ("keane", bf.Keane(), -0.675),
    ("picheny goldstein and price", bf.PichenyGoldsteinAndPrice(), None),
    ("mc cormick", bf.McCormick(), None),
    ("michalewicz", bf.Michalewicz(), None),
    ("styblinski and tang", bf.StyblinskiTang(), None),
    ("easom", bf.Easom(), None),
    ("pits and holes", bf.PitsAndHoles(), None),
    ("egg holder", bf.EggHolder(), None),
]


def f(fni, history):
    x = OptFun(fni)
    x.history = history
    return x

In [ ]:
def ga() -> None:
    for (name, fn, minima) in bfs:

        m = 10
        primes = [2, 3, 5, 7]
        boundaries = OptFun(fn).bounds()
        pop = m ** len(boundaries)
        gs: list[tuple[str, np.ndarray[np.ndarray[float]]]] = [
            ('uniform', random_generator(boundaries, pop, np.random.default_rng(seed=0))),
            ('lhs', lhs_generator(boundaries, pop, np.random.default_rng(seed=0))),
            (f'halton : {primes[:len(boundaries)]}', halton_generator(boundaries, pop, primes[:len(boundaries)])),
            ('full factorial', ff_generator(boundaries, m)),
        ]

        for (g, ps) in gs:
            ps = ps.tolist()
            func = OptFun(fn)

            args = dict(
                num_vars=2,  # number of dimensions of the search space
                gaussian_stdev=1.0,  # standard deviation of the Gaussian mutations
                tournament_size=2,
                num_elites=1,  # number of elite individuals to maintain in each gen
                pop_size=len(ps),  # population size
                seeds=ps,
                # pop_init_range=func.bounds()[0],  # range for the initial population
                max_generations=3 ** 4 - 1,  # number of generations of the GA
                crossover_rate=0.9,
                mutation_rate=0.1,
            )

            run_ga(
                NumpyRandomWrapper(0),  # seeded random number generator
                func,
                **args
            )

            # list(
            #     func.history
            #     | p.Pipe(lambda xs: more_itertools.windowed(xs, n=args['pop_size'], step=args['pop_size']))
            #     | p.map(lambda x: f(fn, x))
            #     | p.izip(it.count(0))
            #     | p.map(lambda x: x[0].heatmap(file=f'out/heatmap {name} {x[1]}.svg'))
            # )

            list(
                range(1, 5)
                | p.map(lambda x: args['pop_size'] * (3 ** x))
                | p.map(lambda x: f(fn, func.history[:x]))
                | p.izip(it.count(0))
                | p.map(lambda x: x[0].heatmapP(
                    init=args['pop_size'],
                    window=args['pop_size'],
                    file=f'ga/heatmap {g} {name} {x[1]}.svg')
                        )
            )

            func.plot(minima=minima, file=f'ga/trend {g} {name}.svg')


shutil.rmtree('ga')
Path('ga').mkdir(parents=True, exist_ok=False)
ga()

In this lab, you will need to compare the algorithms studied in the previous lessons with their version enhanced with different DOE techniques.

1.   How do the performances increases? Are the algorithms faster to converge, or can they find better solutions? 
2.   Is there an approach better than the others in terms of performance?
3.   How much do the DOEs affect the search cost? 
